<a href="https://colab.research.google.com/github/jeudicode/MCC-AP/blob/master/tareas/tarea1/Tarea1_Diego_Isla_Lopez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn import datasets

**Ejercicio 1: Red de unidades de umbral lineal**

Programa y evalúa una red de neuronas con funciones de activación escalón unitario que aproxime la operación XNOR.



In [21]:
def escalon(z):
    if z > 0.0:
        return 1.0
    else:
        return 0.0

In [22]:

def neurona(x, w, b):
  z = np.dot(w.T, x) + b
  a = escalon(z)

  return a

In [23]:
def multicapa(x, W1, b1, W2, b2):
  escv = np.vectorize(escalon)
  a = escv(np.dot(W1.T, x) + b1)
  return escv(np.dot(W2.T, a) + b2)

Definimos el vector objetivo para la compuerta XNOR de la siguiente manera:

In [24]:
y_xnor = [1., 0., 0., 1.]

Definimos el vector con las entradas posibles:

In [25]:
X = np.array([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])


Para la estructura de la red, utilizamos una muy similar a la requerida para la compuerta XOR. En este caso, en lugar de usar una neurona que aplique la operación NAND, utilizamos una que aplique la operación NOR, además de una neurona que aplique la operación OR en la última capa. 

Para esto, primero debemos obtener los pesos necesarios para la operación NOR y definir la salida esperada. En este caso, observamos que los pesos necesarios son -1 para ambas entradas con un sesgo de 1

In [26]:
W1_nor = np.array([-1, -1])
b1_nor = np.array([1])
y_nor = np.array([1.,0.,0.,0.])

In [27]:
for i in range(X.shape[0]):
  y_hat = neurona(X[i], W1_nor, b1_nor)
  print('{0}\t{1}\t{2}\t{3}'.format(X[i, 0], X[i, 1], y_nor[i], y_hat))

0.0	0.0	1.0	1.0
0.0	1.0	0.0	0.0
1.0	0.0	0.0	0.0
1.0	1.0	0.0	0.0


De este modo, sabemos que los pesos y sesgos para las compuertas necesarias son:

AND: W = [10, 10], b = -15

NOR: W = [-1, -1], b = 1

OR:  W = [10, 10], b = -5


Representamos esto en la estructura de la red:

In [38]:
W1 = np.array([[10, -1], [10, -1]])
b1 = np.array([-15, 1])

W2 = np.array([[10],[10]])
b2 = np.array([-5])

Se realiza la evaluación:

In [39]:
for i in range(X.shape[0]):
  y_hat = multicapa(X[i], W1, b1, W2, b2)
  print('{0}\t{1}\t{2}\t{3}'.format(X[i, 0], X[i, 1], y_xnor[i], y_hat[0]))

0.0	0.0	1.0	1.0
0.0	1.0	0.0	0.0
1.0	0.0	0.0	0.0
1.0	1.0	1.0	1.0
